In [17]:
import sys
import time
import random
import math
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import networkx as nx

from copy import deepcopy
from typing import Union, List, Tuple, Dict
from pgmpy.readwrite import XMLBIFReader

from BayesNet import *
from BNReasoner import *
from utils import *

# Initialize
---

In [256]:
# BIFXML files
net = './testing/dog_problem.BIFXML'

In [257]:
# constructs a BN object
bn = BayesNet()

# Loads the BN from an BIFXML file
bn.load_from_bifxml(net)

# create reasoner
bnr = BNReasoner(bn)

### PERFORMANCE EVALUATION

In [288]:
data = []

In [297]:
evi = {
        'dog-out': 'true'
}

qry = ['hear-bark']

start = time.time()
MAP = bnr.MAP(query=qry, evidence=evi, ordering_function=minfill_order)
end = time.time()

print(f"MAP done in {round(end - start, 3)} seconds!\n")

print(f"MAP:\n{MAP}\n")

data.append(('MAP', 5, 'minfill', qry, evi, end-start))
data

MAP done in 0.077 seconds!

MAP:
{'f_mult(f_mult(family-out-f_mult(hear-bark-f_mult(bowel-problem-dog-out)_sum(bowel-problem))_sum(dog-out))_sum(family-out))_max(hear-bark)': p                     0
inst_hear-bark    False
Name: 0, dtype: object}



[('MAP', 5, 'random', ['hear-bark'], {'dog-out': 'true'}, 0.09596943855285645),
 ('MAP', 5, 'random', ['hear-bark'], {'dog-out': 'true'}, 0.09198141098022461),
 ('MAP', 5, 'random', ['hear-bark'], {'dog-out': 'true'}, 0.07597470283508301),
 ('MAP', 5, 'mindeg', ['hear-bark'], {'dog-out': 'true'}, 0.07297635078430176),
 ('MAP', 5, 'mindeg', ['hear-bark'], {'dog-out': 'true'}, 0.07098817825317383),
 ('MAP', 5, 'mindeg', ['hear-bark'], {'dog-out': 'true'}, 0.07100558280944824),
 ('MAP', 5, 'minfill', ['hear-bark'], {'dog-out': 'true'}, 0.0779869556427002),
 ('MAP',
  5,
  'minfill',
  ['hear-bark'],
  {'dog-out': 'true'},
  0.07700419425964355),
 ('MAP',
  5,
  'minfill',
  ['hear-bark'],
  {'dog-out': 'true'},
  0.07698607444763184)]

In [188]:
data.pop()

('MAP',
 11,
 'minfill',
 ['Blurry Vision'],
 {'Headache': True, 'Diabetes': False, 'Sleep Deprivation': True},
 1.4665625095367432)

In [298]:
df = pd.DataFrame(data, columns=["Method", "Vars", "Ordering", "Query", "Evidence", "Runtime"])
df

,Method,Vars,Ordering,Query,Evidence,Runtime
0,MAP,5,random,[hear-bark],{'dog-out': 'true'},0.095969
1,MAP,5,random,[hear-bark],{'dog-out': 'true'},0.091981
2,MAP,5,random,[hear-bark],{'dog-out': 'true'},0.075975
3,MAP,5,mindeg,[hear-bark],{'dog-out': 'true'},0.072976
4,MAP,5,mindeg,[hear-bark],{'dog-out': 'true'},0.070988
5,MAP,5,mindeg,[hear-bark],{'dog-out': 'true'},0.071006
6,MAP,5,minfill,[hear-bark],{'dog-out': 'true'},0.077987
7,MAP,5,minfill,[hear-bark],{'dog-out': 'true'},0.077004
8,MAP,5,minfill,[hear-bark],{'dog-out': 'true'},0.076986


In [299]:
# df.to_csv("outputs/map_5_q3.csv", sep='\t', index=False)

# MPE

In [326]:
data = []

In [327]:
evi = {'dog-out': 'true'}

start = time.time()
MPE = bnr.MPE(evidence=evi, ordering_function=random_order)
end = time.time()

print(f"MPE done in {round(end - start, 3)} seconds!\n")

print(f"MPE:\n{MPE}")

data.append(('MPE', 5, 'minfill', evi, end-start))
data

MPE done in 0.098 seconds!

MPE:
{'f_mult(family-out-f_mult(light-on)_max(light-on)-f_mult(f_mult(bowel-problem-dog-out)_max(bowel-problem)-f_mult(hear-bark)_max(hear-bark))_max(dog-out))_max(family-out)': inst_light-on      False
p                      0
inst_dog-out       False
inst_family-out    False
Name: 0, dtype: object}


[('MPE', 5, 'minfill', {'dog-out': 'true'}, 0.09796857833862305)]

In [321]:
data.pop()

('MPE', 5, 'minfill', {'hear-bark': 'false'}, 0.08695816993713379)

In [324]:
df = pd.DataFrame(data, columns=["Method", "Vars", "Ordering", "Evidence", "Runtime"])
df

,Method,Vars,Ordering,Evidence,Runtime
0,MPE,5,random,{'hear-bark': 'false'},0.111974
1,MPE,5,random,{'hear-bark': 'false'},0.090983
2,MPE,5,random,{'hear-bark': 'false'},0.138986
3,MPE,5,mindeg,{'hear-bark': 'false'},0.090970
4,MPE,5,mindeg,{'hear-bark': 'false'},0.088977
5,MPE,5,mindeg,{'hear-bark': 'false'},0.087984
6,MPE,5,minfill,{'hear-bark': 'false'},0.087000
7,MPE,5,minfill,{'hear-bark': 'false'},0.084002
8,MPE,5,minfill,{'hear-bark': 'false'},0.092970


In [325]:
df.to_csv("outputs/mpe_5_q2.csv", sep='\t', index=False)

# Usage
---

In [ ]:
# query inputs
# evidence = {'hear-bark': True, 'family-out': True}
# x = ['light-on']
# prio = ['dog-out']

### D-separation
Checks whether x and y are d-separated, given (possibly empty) evidence. 

* x and y must be lists
* evidence is (always) a dict
* output is boolean

In [ ]:
# dsep = bnr.d_separated(prio, x, evidence)
# print(f"Are {prio} and {x} d-separated by evidence '{evidence.keys()} = {evidence.values()}'?\n\n{dsep}")

### Ordering functions for determining elimination order
inputs should be network structure
* y = list of query variables
* z = priority variables which are ordered first __[optional]__

In [ ]:
# pi1 = mindeg_order(net=bn, query=prio)
# pi2 = minfill_order(net=bn, query=prio)
# print(f"Variable orderings mindeg:\t{pi1}")
# print(f"Variable orderings minfill:\t{pi2}")

### Marginal distributions
with optional ordering function (name of function is input)

In [ ]:
# without evidence:
# prior_marginal = bnr.compute_marginal(query=x, ordering_function=random_order)
# print(f"\n\nPrior Marginal for Query {x}\n{prior_marginal}")

# with evidence:
# posterior_marginal = bnr.compute_marginal(query=x, evidence=evidence, ordering_function=minfill_order)
# print(f"\nPosterior Marginal for Query {x} with Evidence {evidence}\n{posterior_marginal}")

### Most likely instantiations

In [ ]:
# MAP:
# MAP = bnr.MAP(query=x, evidence=evidence)
# print(f"MAP:\n{[MAP[key] for key in MAP]}\n")

# MPE:
# MPE = bnr.MPE(evidence=evidence, ordering_function=minfill_order)
# print(f"MPE:\n{[MPE[key] for key in MPE]}")